# COVID-19 mobility restriction graphs characterization using TRIPS


### Code contributors:
- Jordi Grau <jordi.grau@eurecat.org>

### Content
This notebook finds differences in Catalunya's mobilty across the previously defined COVID-19 phases using network science techniques. In this notebook the focus is in the numer trips graphs.   

### Contents

1. [Data Loading](#data)   
    1.1. [Daily fluxes queries](#df)   
    1.2. [Low trips edges removal](#removal)   
    1.3. [Region geometries, centroids and geographical distances](#geography-data)


2. [Graphs creation for each COVID-19 phase](#graphs)


3. [Graph analysis](#graph-analysis)  
    3.1. [Mobility map](#mobility-map)   
    3.2. [Degree centrality and density](#degree-density)      
    3.3. [In-degree and out-degree centralities](#in-out)   
    3.4. [Distance probability](#distance-prob)   
    3.5. [Shortest Path and their Lengths $L_{ij}$](#shortest-paths)   
    3.6. [Shortest Path trees](#shortest-paths-trees)   
    3.7. [Closeness centrality](#closness)   
    3.8. [Betweenness centraliy](#betweenness)   
    3.9. [Transitivity and Clustering Coefficient ($C_{ij}$)](#clustering)      

In [ ]:
# Imports
import pickle
from os.path import exists
import numpy as np
import geopandas
import matplotlib.pyplot as plt

import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from sklearn import linear_model

from src.mobility_context_and_queries import *

# URL to save processed dataframes and graphs
save_url = '../../jupyter/Observatori/data/processed/graphs and metrics/trips/'

# COVID-19 phases
phases_names = [list(i.keys())[0] for i in phases_list]
phases_list

# 1. Data Loading <a class="anchor" id="data"></a>

## 1.1. Daily fluxes queries <a class="anchor" id="df"></a>

In [ ]:
# DAILY FLUX query between 2 dates of trips from or to Catalunya 
date1 = phases_list[0]['precovid']['start']
date2 = phases_list[9]['nadal']['end']
province_group = list(location_info.items())

df_flux_daily = query_raw_data_or_trips_or_flux_matrix_between_dates(table='mitma_trips_matrix', date1=date1, date2=date2, province_groups=province_group)
df_flux_daily['datetime'] = pd.to_datetime(df_flux_daily['datetime'], format='%Y-%m-%m')
df_flux_daily['source'] = df_flux_daily['source'].astype(str)
df_flux_daily['target'] = df_flux_daily['target'].astype(str)

# Exclude nodes outside of Catalunya
df = pd.DataFrame()
for mg in province_group:
    postal_code = mg[1][0]
    df = df.append(df_flux_daily[(df_flux_daily['source'].str[:2] == postal_code)])
df_flux_daily = df

source_nodes = list(df_flux_daily['source'].unique())
df_flux_daily = df_flux_daily[df_flux_daily['target'].isin(source_nodes)]

# Remove the dot that indicates thousands in the trips column
df_flux_daily['trips'] = df_flux_daily['trips'].apply(round)

# Describe dataframe
print(f"df_flux_daily size: {df_flux_daily.size}")
df_flux_daily.head(2)

In [ ]:
# Create nodes dataframe (source and source_labels)
all_nodes = list(df_flux_daily['source'].unique())
df_nodes = pd.DataFrame(all_nodes)
df_nodes.reset_index(inplace=True)
df_nodes.columns = ['source', 'source_label']

# Add source nodes to flux dataframe
df_flux_daily = pd.merge(
    df_nodes, df_flux_daily, how="inner",
    left_on='source_label', right_on='source', suffixes=("", "_y"))
df_flux_daily.drop(['source_y'], axis=1, inplace=True)

# Add target nodes to flux dataframe
df_nodes = df_nodes.rename(columns={'source': 'target', 'source_label': 'target_label'})
df_flux_daily = pd.merge(
    df_nodes, df_flux_daily, how="inner", 
    left_on='target_label', right_on='target', suffixes=("", "_y"))
df_flux_daily.drop(['target_y'], axis=1, inplace=True)

# Reorder columns
df_flux_daily = df_flux_daily[['datetime', 'source', 'source_label', 'target', 'target_label', 'trips']]
df_flux_daily = df_flux_daily.sort_values(['source', 'datetime', 'target'])
df_flux_daily.head(2)

## 1.2. Low trips edges removal <a class="anchor" id="removal"></a>

In [ ]:
# Study trips distribution
df_flux_daily['trips'].hist(bins=100)
plt.show()
pd.DataFrame(df_flux_daily['trips'].quantile([0.05, 0.2, 0.4, 0.6, 0.8, 0.9, 0.95, 1])).T

In [ ]:
# Filter 5% of edges with lower number of trips
df_flux_daily = df_flux_daily[df_flux_daily['trips'] >= 5]

## 1.3. Region geometries, centroids and geographical distances  <a class="anchor" id="geography-data"></a>

In [ ]:
# Get geometry info from qrp indexes
df_qrp = query_trips_iio_or_qrp_between_dates(table='mitma_qrp', province_groups=province_group)
df_geometry = df_qrp[['source', 'geometry']].reset_index(drop=True).drop_duplicates()

# Get centroids
df_geometry['centroid'] = df_geometry['geometry'].centroid

# Add node's names and labels to geometry dataframe
df_geometry.columns = ['source_label', 'geometry', 'centroid']
df_nodes.columns = ['source', 'source_label']

df_geometry = pd.merge(
    df_nodes, df_geometry, how="inner",
    left_on='source_label', right_on='source_label')
df_geometry.head(2)

In [ ]:
# Compute geographical distances between two centroids
a = gpd.GeoSeries(df_geometry['centroid'], crs="EPSG:4326")
b = a.copy()
centroid_distances = a.apply(lambda g: b.distance(g))

# Matrix to dataframe 
df_distances = pd.melt(centroid_distances.reset_index(), id_vars=['index'], var_name='description')
df_distances.columns = ['source', 'target', 'geo_distance']
df_distances['geo_distance'] = round(df_distances['geo_distance']/1000, 3)

# Distance binning
labels = ['<10', '10-50', '50-200', '200-600']
bins = [0, 10, 50, 200, 600]
df_distances['geo_distance_b'] = pd.cut(df_distances['geo_distance'], bins=bins, labels=labels, right=True)
df_distances.head(2)

# 2. Graphs creation for each COVID-19 phase <a class="anchor" id="graphs"></a>

In [ ]:
def compute_inverse_trips(df):
    """
    Compute inverse of number of trips
    """    
    # Inverse p
    df['inverse_trips'] = 1 / df['trips']
    
    return df

In [ ]:
def create_digraph(df, df_geometry, df_distances):
    """
    Creates directed graph from a dataframe with source, target, trips, inverse_trips and total_trips columns
    """
    # Add nodes
    G = nx.from_pandas_edgelist(df, source='source', target='target', 
        edge_attr=['trips', 'inverse_trips'], create_using=nx.DiGraph)

    # Add nodes geometries, centroids and labels
    for index, row in df_geometry.iterrows():
        G.nodes[row['source']]['geometry'] = row['geometry']
        G.nodes[row['source']]['centroid'] = row['centroid']
        G.nodes[row['source']]['label'] = row['source_label']

    # Add edges geographical distances
    for index, row in df_distances.iterrows():
        # Graphs don't have edges with same source and target
        #if row['source'] ==  row['target']:
        #    continue
        # Graphs don't have all combinations of source and target
        try:
            G.edges[row['source'], row['target']]['geo_distance'] = row['geo_distance']
            G.edges[row['source'], row['target']]['geo_distance_b'] = row['geo_distance_b']
        except:
            continue
    
    return G

In [ ]:
def load_or_compute_dataframe_and_graph(df, covid_phase, save_url, df_geometry, df_distances, compute=False):
    """
    Loads or computes the dataframes and graphs of the different covid phases. 
    Also dataframes and graphs are saved if doesn't exist the file.
    """
    
    # Dataframe
    if (exists(f'{save_url}/{phases_names[covid_phase]}_df.csv')) & (compute == False):
        df = pd.read_csv(f'{save_url}/{phases_names[covid_phase]}_df.csv')
    else:
        df = compute_inverse_trips(df)
        df.to_csv(f'{save_url}/{phases_names[covid_phase]}_df.csv', index=False)
    
    # Graph
    if (exists(f'{save_url}/{phases_names[covid_phase]}_graph.gpickle')) & (compute == False):
        G = nx.read_gpickle(f'{save_url}/{phases_names[covid_phase]}_graph.gpickle')
    else:
        G = create_digraph(df, df_geometry, df_distances)
        nx.write_gpickle(G, f'{save_url}/{phases_names[covid_phase]}_graph.gpickle')
        
    # Describe Graph
    print(f'Number of nodes: {G.number_of_nodes()}')
    print(f'Number of edges: {G.number_of_edges()}\n')
    
    return df, G

In [ ]:
# Get dataframes for each covid phases and compute mean p mobility index
df_precovid = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[0]['precovid']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[0]['precovid']['end'  ])]
df_precovid = df_precovid.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_lockdown = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[1]['lockdown']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[1]['lockdown']['end'  ])]
df_lockdown = df_lockdown.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_mobilitat_essenc = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[2]['mobilitat_essenc']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[2]['mobilitat_essenc']['end'  ])]
df_mobilitat_essenc = df_mobilitat_essenc.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_fase_0 = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[3]['fase_0']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[3]['fase_0']['end'  ])]
df_fase_0 = df_fase_0.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_desescalada = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[4]['desescalada']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[4]['desescalada']['end'  ])]
df_desescalada = df_desescalada.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_no_restriccions = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[5]['no_restriccions']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[5]['no_restriccions']['end'  ])]
df_no_restriccions = df_no_restriccions.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_alerta_5_inici = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[6]['alerta_5_inici']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[6]['alerta_5_inici']['end'  ])]
df_alerta_5_inici = df_alerta_5_inici.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_alerta_5_tr1 = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[7]['alerta_5_tr1']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[7]['alerta_5_tr1']['end'  ])]
df_alerta_5_tr1 = df_alerta_5_tr1.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_alerta_5_tr1_2 = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[8]['alerta_5_tr1_2']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[8]['alerta_5_tr1_2']['end'  ])]
df_alerta_5_tr1_2 = df_alerta_5_tr1_2.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

df_nadal_cap_reis = df_flux_daily[
    (df_flux_daily['datetime'] >= phases_list[9]['nadal']['start']) & 
    (df_flux_daily['datetime'] <= phases_list[9]['nadal']['end'  ])]
df_nadal_cap_reis = df_nadal_cap_reis.groupby(['source', 'source_label', 'target', 'target_label']).mean().reset_index()

In [ ]:
# Load or compute dataframes and graphs for each phase
print("Precovid")
df_precovid_mean, G_precovid = load_or_compute_dataframe_and_graph(
    df_precovid, 0, save_url, df_geometry, df_distances)

print("Lockdown")
df_lockdown_mean, G_lockdown = load_or_compute_dataframe_and_graph(
    df_lockdown, 1, save_url, df_geometry, df_distances)

print("Mobilitat essencial")
df_mobilitat_mean, G_mobilitat = load_or_compute_dataframe_and_graph(
    df_mobilitat_essenc, 2, save_url, df_geometry, df_distances)

print("Fase 0")
df_fase0_mean, G_fase0 = load_or_compute_dataframe_and_graph(
    df_fase_0, 3, save_url, df_geometry, df_distances)

print("Desescalada")
df_desescalada_mean, G_desescalada = load_or_compute_dataframe_and_graph(
    df_desescalada, 4, save_url, df_geometry, df_distances)

print("No restriccions")
df_no_restriccions_mean, G_no_restriccions = load_or_compute_dataframe_and_graph(
    df_no_restriccions, 5, save_url, df_geometry, df_distances)

print("Alerta 5 inici")
df_alert5_mean, G_alert5 = load_or_compute_dataframe_and_graph(
    df_alerta_5_inici, 6, save_url, df_geometry, df_distances)

print("Alerta 5 inici tram 1")
df_alert5_tr1_mean, G_alert5_tr1 = load_or_compute_dataframe_and_graph(
    df_alerta_5_tr1, 7, save_url, df_geometry, df_distances)

print("Alerta 5 inici tram 1__2")
df_alert5_tr2_mean, G_alert5_tr2 = load_or_compute_dataframe_and_graph(
    df_alerta_5_tr1_2, 8, save_url, df_geometry, df_distances)

print("Nadal, cap d'any, reis mags")
df_xmas, G_xmas = load_or_compute_dataframe_and_graph(
    df_nadal_cap_reis, 9, save_url, df_geometry, df_distances)

# 3. Graphs analysis <a class="anchor" id="analysis"></a>

In [ ]:
# Prepare lists with study df, graphs and titles
list_titles = [
    'Precovid', 'Lockdown', 'Mobilitat essencial', 
    'Fase 0', 'Desescalada', 'No restriccions', 
    'Alerta 5 inici', 'Alerta 5 tram 1', 'Alerta 5 flexible', 
    'Nadal, cap d\'any, reis mags']

list_df = [
    df_precovid_mean, df_lockdown_mean, df_mobilitat_mean, 
    df_fase0_mean, df_desescalada_mean, df_no_restriccions_mean,
    df_alerta_5_inici, df_alerta_5_tr1, df_alerta_5_tr1_2, 
    df_nadal_cap_reis]

list_G = [
    G_precovid, G_lockdown, G_mobilitat, G_fase0, 
    G_desescalada, G_no_restriccions, G_alert5, G_alert5_tr1, 
    G_alert5_tr2, G_xmas]

print(len(list_titles), len(list_df), len(list_G))

In [ ]:
### Filter graph by geographical distance edge attribute
# Get edges with each distance label 
edges_by_distance = {}
for label in labels:
    nodes = []
    for source, target, edge_attr in G_precovid.edges(data=True):
        if edge_attr['geo_distance_b'] == label:
            nodes.append((source, target))
    edges_by_distance[label] = nodes

In [ ]:
def filter_graphs_by_distance(G, edges_by_distance):
    """
    Creates 4 graphs grouping the nodes which have a certain edge attribute 
    distance value. 
    """
    G_10  = G.edge_subgraph(edges_by_distance['<10'])
    G_50  = G.edge_subgraph(edges_by_distance['10-50'])
    G_200 = G.edge_subgraph(edges_by_distance['50-200'])
    G_600 = G_precovid.edge_subgraph(edges_by_distance['200-600'])
    
    return G_10, G_50, G_200, G_600

## 3.1. Mobility map <a class="anchor" id="mobility-map"></a>

In [ ]:
max(df_flux_daily['trips'])

In [ ]:
def plot_graph_map_mobility(G, df_geometry, title):
    
    # Get edge normalized weights
    nodes = G.nodes
    edges = [G[u][v] for u,v in G.edges]
    weights = [i['trips']/324985 for i in edges]

    # Get node positions
    gpd_geometry = gpd.GeoDataFrame(df_geometry, crs="EPSG:4326", geometry='geometry')
    gpd_geometry["centroid"] = gpd_geometry["geometry"].centroid
    gpd_geometry["x"] = gpd_geometry.centroid.map(lambda p: p.x)
    gpd_geometry["y"] = gpd_geometry.centroid.map(lambda p: p.y)
    centroids = np.column_stack((gpd_geometry['x'], gpd_geometry['y']))
    positions = dict(zip(nodes, centroids))

    # Plot
    fig, ax = plt.subplots(figsize=(10,10))
    gpd_geometry['geometry'].plot(ax=ax, edgecolor='white', alpha=0.4)
    nx.draw(
        G.subgraph(nodes), pos=positions,
        node_size=4, width=weights, arrows=False,         
        edge_vmin=0, edge_vmax=1)
    
    fig.suptitle(f'{title}', size=16, y=0.85)
    
    plt.show()
    
    print(f"Sum of mean trips in {title}: {round(np.sum([i['trips'] for i in edges]))},  " + 
          f"mean: {round(np.mean([i['trips'] for i in edges]))}")

In [ ]:
for G, title in zip(list_G, list_titles):
    plot_graph_map_mobility(G, df_geometry, title)

## 3.2. Degree centrality and density <a class="anchor" id="degree-density"></a>

In [ ]:
# Nodes with more degrees and distribution
degrees = pd.DataFrame(G_precovid.degree(), columns=['node','degree'])
degrees['degree'].hist()
plt.show()

# MITMA regions and number of degrees
print(df_nodes[df_nodes['source'].isin([15, 17, 22])])
degrees[degrees['node'].isin([15, 17, 22])]

In [ ]:
def plot_degree_histogram(list_G, list_titles, edges_by_distance, by_distance=False):
    
    fig, axs = plt.subplots(5,2,  figsize=(14, 9), sharey=True, sharex=True)
    
    # For each graph
    row = 0
    column = 0
    for i, G in enumerate(list_G):
        
        not_modified = True
        
        # Plot degrees
        degrees = [G.degree(n) for n in G.nodes()]
        pd.DataFrame(degrees).hist(ax=axs[column][row])
        
        # Compute average degree and density
        average = np.mean(degrees)
        density = nx.density(G)
        axs[column][row].set_title(f"{list_titles[i]}: {round(density, 3)} density")
        
        # Plot average
        axs[column][row].axvline(x=average, color='b', label='average degree')
        
        # Rows and columns update
        if row == 0:
            row += 1
            not_modified = False
        
        if (row == 1) & not_modified:
            column += 1
            row = 0
        
    fig.suptitle(f'Degree frequencies', size=16, y=1)
    plt.tight_layout()
    
    # For each graph 
    if by_distance:
        fig, axs = plt.subplots(len(list_G), 4, figsize=(14, 15), sharey=True, sharex=True)
        fig.suptitle(f'Degree frequencies', size=16, y=1)
        
        for i, G in enumerate(list_G):
            # Filter nodes by distance
            G_10, G_50, G_200, G_600 = filter_graphs_by_distance(G, edges_by_distance)

            pd.DataFrame([G_10.out_degree(n) for n in G_10.nodes()]).hist(ax=axs[i][0], bins=10)
            pd.DataFrame([G_50.out_degree(n) for n in G_50.nodes()]).hist(ax=axs[i][1], bins=10)
            pd.DataFrame([G_200.out_degree(n) for n in G_200.nodes()]).hist(ax=axs[i][2], bins=10)
            pd.DataFrame([G_600.out_degree(n) for n in G_600.nodes()]).hist(ax=axs[i][3], bins=10)
            
            for col in range(4):
                axs[i][col].set_title('')
            axs[i][0].set_ylabel(f'{list_titles[i]}')

        # Set title
        axs[0][0].set_title(f'{labels[0]} Km')
        axs[0][1].set_title(f'{labels[1]} Km')
        axs[0][2].set_title(f'{labels[2]} Km')
        axs[0][3].set_title(f'{labels[3]} Km')
            

        plt.tight_layout()
        plt.show()

In [ ]:
plot_degree_histogram(list_G, list_titles, edges_by_distance, by_distance=True)

## 3.3. In-degree and Out-degree centralities <a class="anchor" id="in-out"></a>

In [ ]:
# In/Out-degree of a node
target_mitma = '0801903'
SOURCE = df_nodes[df_nodes['source_label'] == target_mitma]['source'].values[0]

for metric in metrics:
    print("\n", metric)
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        in_degree = G.in_degree(nbunch=[SOURCE])
        out_degree = G.out_degree(nbunch=[SOURCE])
        print("\t", title,":", in_degree[0],"/", out_degree[0])

In [ ]:
def plot_in_degree_histogram(list_G, list_titles, by_distance=False):
    
    fig, axs = plt.subplots(len(list_G),2,  figsize=(2*len(list_G),3*len(list_G)))
    
    # For each graph
    for i, G in enumerate(list_G):
        
        # Plot in_degrees
        in_degrees = [G.in_degree(n) for n in G.nodes()]
        pd.DataFrame(in_degrees).hist(ax=axs[i][0])
        average = np.mean(in_degrees)
        density = nx.density(G)
        axs[i][0].set_title(f"{list_titles[i]}: {round(density, 3)} density")
        
        # Plot average
        axs[i][0].axvline(x=average, color='b', label='average degree')
        
        # Plot out_degrees
        out_degrees = [G.out_degree(n) for n in G.nodes()]
        pd.DataFrame(out_degrees).hist(ax=axs[i][1])
        average = np.mean(out_degrees)
        density = nx.density(G)
        axs[i][1].set_title(f"{list_titles[i]}")
        
        # Plot average
        axs[i][1].axvline(x=average, color='b', label='average degree')
        
    fig.suptitle(f'In-Degree and Out-Degree frequencies', size=16, y=1)
    plt.tight_layout()

In [ ]:
plot_in_degree_histogram(list_G[:5], list_titles[:5], by_distance=False)
plot_in_degree_histogram(list_G[5:], list_titles[5:], by_distance=False)

## 3.4. Distance probability <a class="anchor" id="distance-prob"></a>

In [ ]:
def plot_distance_probability(list_G, list_titles):
    fig, axs = plt.subplots(5,2, figsize=(14,9), sharey=True, sharex=True)
    
    column = 0
    row = 0
    
    # For each graph
    for i, G in enumerate(list_G):
        # Row and columns update
        not_modified = True
        
        edges = [G[u][v] for u,v in G.edges]
        weights = [i['geo_distance'] for i in edges]
        axs[column][row].hist(weights, bins=100, histtype='step', density=True, cumulative=-1)

        axs[column][row].set_title(f"{list_titles[i]}")
        axs[4][row].set_xlabel("Km")
        axs[column][0].set_ylabel("P(d_ij >= d)")
        
        axs[column][row].set_yticks([0, 0.25, 0.5, 0.75, 1])
        axs[column][row].grid()
        
        # Rows and columns update
        if row == 0:
            row += 1
            not_modified = False
        if (row == 1) & not_modified:
            column += 1
            row = 0
        

    fig.suptitle(f'Distance probability', size=16, y=1)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_distance_probability(list_G, list_titles)

## 3.5. Shortest Path and their Lengths $L_{ij}$ <a class="anchor" id="shortest-paths"></a>

In [ ]:
def compute_shortest_paths(G, metric, weighted=True, compute_paths=False):
    """
    Computes shortest paths of a Graph Network, their lengths, and the average
    shortest path length of the network.
    """
    if weighted:
        if compute_paths:
            # Compute shortest path and their lenghts
            shortest_paths = nx.shortest_path(G, method='dijkstra', weight=metric)
            shortest_path_lengths = list(nx.shortest_path_length(G, method='dijkstra', weight=metric))
        else:
            shortest_paths, shortest_path_lengths = None, None

        # Average shortest path length
        average_shortest_path_length = nx.average_shortest_path_length(G, method='dijkstra', weight=metric)
    
    else:
        if compute_paths:
            # Compute shortest path and their lenghts
            shortest_paths = nx.shortest_path(G)
            shortest_path_lengths = nx.shortest_path_length(G)
        else:
            shortest_paths, shortest_path_lengths = None, None

        # Average shortest path length
        average_shortest_path_length = nx.average_shortest_path_length(G)
        
    return average_shortest_path_length, shortest_paths, shortest_path_lengths

In [ ]:
def path_and_lengths_to_dataframe(shortest_paths, shortest_path_lengths):
    """
    Creates a dataframe from merging shortest_paths and the shortest_path_lengths 
    originated from a Graph using the library Networkx.
    """
    
    # Shortest paths
    df_shortest_paths = pd.DataFrame()
    for source, target_path in enumerate(shortest_paths.items()):
        for target, path in target_path[1].items():
            df_shortest_paths = df_shortest_paths.append([[source, target, path]])
    df_shortest_paths.columns = ['source', 'target', 'path']

    # Shortest path lengths
    df_shortest_path_lengths = pd.DataFrame()
    for source, target_distance_dict in shortest_path_lengths:
        for target, distance in target_distance_dict.items():
            df_shortest_path_lengths = df_shortest_path_lengths.append([[source, target, distance]])
    df_shortest_path_lengths.columns = ['source', 'target', 'distance']
    
    # Merge dataframes
    df_shortest_paths_and_path_lengths = pd.merge(df_shortest_paths, df_shortest_path_lengths, how="inner")
    
    return df_shortest_paths_and_path_lengths

In [ ]:
# Compute shortests paths and their lengths
METRICS = ['inverse_trips', 'geo_distance']

for METRIC in METRICS:

    # Compute all paths and their lengths
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        print(title)

        if exists(f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{METRIC}.csv'):
            continue
        else:
            # Compute shortest paths and their lengths
            average_len, shortest_paths, shortest_path_lengths = compute_shortest_paths(
                G, METRIC, weighted=True, compute_paths=True)

            # Paths and lengths to csv
            df_shortest_paths_and_path_lengths = path_and_lengths_to_dataframe(
                shortest_paths, shortest_path_lengths)
            df_shortest_paths_and_path_lengths.to_csv(
                f'{save_url}{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{METRIC}.csv',
                index=False)

In [ ]:
def plot_shortest_paths(df, title, metric):
    
    fig, axs = plt.subplots(1,3, figsize=(15,5))
    
    # Single variables
    df['len'].hist(ax=axs[0])
    df['distance'].hist(ax=axs[1])
    
    # Plot vertical lines
    axs[0].axvline(x=df['len'].mean(), color='b')
    axs[1].axvline(x=df['distance'].mean(), color='b')
    
    # 2 variables linear regression and R^2
    x = df['len'].values.reshape(-1, 1)
    y = df['distance'].values.reshape(-1, 1)
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    plt.plot(x, regr.predict(x), color='red', linewidth=3)
    r2 = round(regr.score(y, x), 2)
    m = round(regr.coef_[0][0], 2)
    
    # 2 variables scatter plot
    df.plot.scatter(x='len', y='distance', ax=axs[2])
    
    if metric == 'inverse_trips':
        # Axis limits
        axs[0].set_xlim([0, 3.5])
        axs[0].set_ylim([0, 180_000])

        axs[1].set_xlim([0, 2.5])
        axs[1].set_ylim([0, 180_000])

        axs[2].set_xlim([0, 2.5])
        axs[2].set_ylim([0, 3.5])
    
    elif metric == 'geo_distance':
        # Axis limits
        axs[0].set_xlim([0, 8])
        axs[0].set_ylim([0, 175_000])

        axs[1].set_xlim([0, 310])
        axs[1].set_ylim([0, 50_000])

        axs[2].set_xlim([0, 10])
        axs[2].set_ylim([0, 320])

    # Set titles
    axs[0].set_title('Shortest path')
    axs[1].set_title('Shortest path length')
    axs[2].set_title(f'Length vs Distance (m: {m}, R2: {r2})')
    
    # x labels
    axs[0].set_xlabel('Number of nodes from source to target')
    
    if metric == 'inverse_trips':
        axs[1].set_xlabel('Sum of 1/trips from source to target')
    elif metric == 'geo_distance':
        axs[1].set_xlabel('Sum of km from source to target')
    
    # Figure title
    fig.suptitle(title, size=16)

    plt.show()

In [ ]:
# Plot shortest paths and lengths' statistics
metrics = ['geo_distance', 'inverse_trips']

for metric in metrics:
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        try:
            df = pd.read_csv(
                f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv', 
                index_col='Unnamed: 0')
        except:
            df = pd.read_csv(
                f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv')

        # Format path column to list
        df['path'] = df['path'].str.replace("[", "")
        df['path'] = df['path'].str.replace("]", "")
        df['path'] = df['path'].str.split(", ")

        # Compute length of the path
        df['len'] = df['path'].apply(len)
        title = f'{title} {metric}: Average Shortest Path {round(df["len"].mean(), 2)}, Length: {round(df["distance"].mean(), 2)}'

        plot_shortest_paths(df, title, metric)

In [ ]:
from fitter import Fitter, get_common_distributions, get_distributions

In [ ]:
# Length and distance distributions across phases
for metric in ['geo_distance', 'inverse_trips']:
    # Create LENGTH's figures
    fig, axs = plt.subplots(1, 10, figsize=(14, 2.5), sharey=True)
    fig.suptitle(f"Shortest path's lengths distributions", size=16)
    
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        # Open file
        try:
            df = pd.read_csv(
                f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv', 
                index_col='Unnamed: 0')
        except:
            df = pd.read_csv(
                f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv')

        # Create len column
        df['path'] = df['path'].str.replace("[", "")
        df['path'] = df['path'].str.replace("]", "")
        df['path'] = df['path'].str.split(", ")
        df['len'] = df['path'].apply(len)

        # Plot
        df['len'].hist(ax=axs[covid_phase], bins=8)
        axs[covid_phase].axvline(x=df['len'].mean(), color='b')
        axs[covid_phase].annotate(
            xycoords='axes fraction', xy=(0.35, 0.9), text=f"mean: {str(round(df['len'].mean(), 2))}", size=8)
        
        # Title and axis
        axs[covid_phase].set_title(title, size=10)
        
        if metric == 'geo_distance':
            axs[covid_phase].set_xticks([0, 2, 4, 6, 8])
        elif (metric == 'inverse_trips') & :
            axs[covid_phase].set_xticks([0, 5, 10, 15, 20, 25])
        
    plt.tight_layout()
    plt.show()
    
    # Create DISTANCE's figures
    fig, axs = plt.subplots(1, 10, figsize=(16, 2.5), sharey=True)
    fig.suptitle(f"Shortest paths' distances", size=16)
    
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        # Open file
        try:
            df = pd.read_csv(
                f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv', 
                index_col='Unnamed: 0')
        except:
            df = pd.read_csv(
                f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv')

        # Plot
        df['distance'].hist(ax=axs[covid_phase])
        axs[covid_phase].set_title(title, size=8)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Create DISTANCE's filtered distributions figures
for metric in ['geo_distance', 'inverse_trips']:
    
    # Create figure
    fig, axs = plt.subplots(5, 10, figsize=(16, 11), sharey='row', sharex=True)
    fig.suptitle(f"Distance distribution by shortest path length value", size=16)
    
    for i in range(1, 5+1):
        
        for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
            # Open file
            try:
                df = pd.read_csv(
                    f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv', 
                    index_col='Unnamed: 0')
            except:
                df = pd.read_csv(
                    f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv')

            # Create len column
            df['path'] = df['path'].str.replace("[", "")
            df['path'] = df['path'].str.replace("]", "")
            df['path'] = df['path'].str.split(", ")
            df['len'] = df['path'].apply(len)
            
            # Filter distance by len
            if i == 5:
                df_filtered = df[df['len'] > 5]
            elif i > 5:
                continue
            else:
                df_filtered = df[df['len'] == i]
            
            # Plot subfigure
            df_filtered['distance'].hist(ax=axs[i-1][covid_phase])
            axs[0][covid_phase].set_title(title, size=10)
            axs[4][covid_phase].set_xlabel('Km')
            axs[i-1][0].set_ylabel(f"Len {i}")
            axs[4][covid_phase].set_xticks([0, 100, 200])
            
            if i == 5:
                axs[i-1][0].set_ylabel(f"Len >={i}")
        
    plt.tight_layout()
    plt.show()

In [ ]:
# Test distance distribution in each phase grouping by len of the path
for metric in ['geo_distance']:
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        print(title)
        try:
            df = pd.read_csv(
                f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv', 
                index_col='Unnamed: 0')
        except:
            df = pd.read_csv(
                f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv')

        # Format path column to list
        df['path'] = df['path'].str.replace("[", "")
        df['path'] = df['path'].str.replace("]", "")
        df['path'] = df['path'].str.split(", ")
        
        # Compute length of the path
        df['len'] = df['path'].apply(len)
        
        for i in range(1, max(df['len'])+1):
            # Filter distance by len path
            if i == 5:
                df_filtered = df[df['len'] > 5]
            elif i > 5:
                continue
            else:
                df_filtered = df[df['len'] == i]

            print(f"Len {i}")

            distances =  df_filtered['distance'].values
            distributions_to_check = [
                'chi2', 'expon', 'exponpow', 'gamma', 'lognorm', 'norm', 'powerlaw']
            f = Fitter(distances, distributions=distributions_to_check)
            f.fit()
            print(f.summary()[:2][['sumsquare_error', 'ks_pvalue']])
            
            plt.show()
            
#             statistic, p_value = shapiro(distances)
#             if p_value > 0.05:
#                 print("NORMAL:", p_value)

## 3.6 Shortest path trees <a class="anchor" id="shortest-paths-trees"></a>

In [ ]:
def split_nodes_in_path_file(SOURCE, covid_phase, metric='geo_distance'):

    # Load shortest paths csv
    try:
        df_geo_dist = pd.read_csv(
            f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv', 
            index_col='Unnamed: 0')
    except:
        df_geo_dist = pd.read_csv(f'{save_url}/{phases_names[covid_phase]}_df_shortest_paths_and_lengths_{metric}.csv')

    # Filter source node
    df = df_geo_dist[(df_geo_dist['source'] == SOURCE) & (df_geo_dist['target'] != SOURCE)]
    df = df[['source', 'target', 'distance', 'path']]

    # Compute max path length
    df['path_list'] = df['path']
    df.loc[:, 'path_list'] = df['path_list'].str.replace(f"\[", "")
    df.loc[:, 'path_list'] = df['path_list'].str.replace("]", "")
    df.loc[:, 'path_list'] = df['path_list'].str.split(", ")
    max_len = max(df['path_list'].apply(len))

    # Extract nodes from list of nodes in paths
    df['path_str'] = df['path'].str.replace(f"\[", "")
    df['path_str'] = df['path_str'].str.replace("\]", "")
    children_nodes = df['path_str'].str.split(', ', expand=True)

    # Create columns names
    new_cols = [f'children_{i+1}' for i in range(0,max_len)]
    all_cols = list(df.columns) + new_cols

    # Add children nodes to dataframe
    df_splitted_paths = pd.concat([df, children_nodes], axis=1)
    df_splitted_paths.columns = all_cols

    for col in new_cols:
        df_splitted_paths[col] = df_splitted_paths[col].apply(str)

    return df_splitted_paths

In [ ]:
def create_digraph_with_one_source_node_using_paths_dataframe(
    SOURCE, metric, df_splitted_paths, df_distances, df_covid_phase):
    
    # Create Digraph with source node
    G = nx.DiGraph()
    G.add_node(SOURCE)
    
    # Hierarchy of nodes
    children_cols = df_splitted_paths.columns[6:]
    
    for i, col in enumerate(children_cols):
        # First nodes after source node
        if col == 'children_1':
            for index, row in df_splitted_paths.iterrows():
                G.add_edge(SOURCE, int(row[col]))

        # Nodes children and their successors
        else:
            for index, row in df_splitted_paths.iterrows():
                if (row[children_cols[i-1]] == 'None') or (row[children_cols[i]] == 'None'):
                    continue
                else:
                    G.add_edge(int(row[children_cols[i-1]]), int(row[children_cols[i]]))    
    
    if metric == 'geo_distance':
        # Add distance between nodes which don't have any node between them to the Graph
        for index, row in df_splitted_paths.iterrows():
            if len(df_splitted_paths.loc[index,'path_list']) == 1:
                G.edges[(row['source'], row['target'])]['len'] = row['distance']

        # Add edges geographical distances
        for index, row in df_distances.iterrows():
            # Graphs don't have all combinations of source and target
            try:
                G.edges[row['source'], row['target']]['len'] = row[metric]
            except:
                continue
    
    elif metric == 'inverse_trips':
        for index, row in df_covid_phase.iterrows():
            # Graphs don't have all combinations of source and target
            try:
                G.edges[row['source'], row['target']]['len'] = row[metric]
            except:
                continue
            
    return G

In [ ]:
def plot_shortest_path_tree(
    SOURCE, metric, covid_phase, df_covid_phase, df_distances, df_nodes, 
    title, place, want_labels=False):
    
    # From a SOURCE node, split shortest_path nodes in different columns and create digraph 
    df_splitted_paths = split_nodes_in_path_file(SOURCE, covid_phase=covid_phase, metric=metric)
    G = create_digraph_with_one_source_node_using_paths_dataframe(
        SOURCE, metric, df_splitted_paths, df_distances, df_covid_phase)
    
    # Add a context edge to allow comparisons between plots
    if metric == 'geo_distance':
        G.add_edge(500, 501, len= 300)
#     elif metric == 'inverse_trips':
#         G.add_edge(500, 501, len= 0.01)

    # Figure 
    fig = plt.figure(dpi=200)
    fig.suptitle(f'{title} in {place}: {metric}')
    
    # Layout
    pos = graphviz_layout(G, prog="neato", root=SOURCE)
    
    # Plot graph
    if want_labels:
        labels = df_nodes.set_index('source')['source_label']
        nx.draw(
            G, pos, ax=fig.add_subplot(111), node_size=3, width=0.1, 
            arrows=True, arrowsize=3, labels=labels, font_size=3, font_color='purple')
    else:
        nx.draw(G, pos, ax=fig.add_subplot(111), node_size=3, width=0.1, arrows=False, arrowsize=3)
    
    # Plot Source node in red
    nx.draw_networkx_nodes(G, pos, nodelist=[SOURCE], node_size=4, node_color='red')

    plt.show()
    
    return df_splitted_paths, G

In [ ]:
# Find Sants Estació and Plaça Catalunya's MITMA districts
bcn_pc = []
for target in df_nodes['source_label'].values:
    if '08019' in target:
        bcn_pc.append(target)
        
for target in bcn_pc:
    SOURCE = df_nodes[df_nodes['source_label'] == target]['source'].values[0]
    fig, ax = plt.subplots(figsize=(5,5))
    fig.suptitle(f"{target}, {SOURCE}")
    a = df_geometry[df_geometry['source_label'] == target]
    b = df_geometry[(df_geometry['source_label'].isin(bcn_pc)) & (df_geometry['source_label'] != target)]
    gpd_geometry_b = gpd.GeoDataFrame(b, crs="EPSG:4326", geometry='geometry')
    gpd_geometry_a = gpd.GeoDataFrame(a, crs="EPSG:4326", geometry='geometry')
    gpd_geometry_a['geometry'].plot(color='red', ax=ax)
    gpd_geometry_b['geometry'].plot(color='blue', ax=ax)
    plt.show()

In [ ]:
# Sants Estació tree
target_mitma = '0801903'
SOURCE = df_nodes[df_nodes['source_label'] == target_mitma]['source'].values[0]
metrics = ['geo_distance', 'inverse_trips']

for metric in metrics:
    for covid_phase, (title, df_covid_phase) in enumerate(zip(list_titles, list_df)):
        df, G = plot_shortest_path_tree(
            SOURCE, metric, covid_phase, df_covid_phase, df_distances, df_nodes, 
            title, 'Sants Estació', want_labels=False)

In [ ]:
# Plaça Catalunya tree
target_pc = '0801901'
SOURCE = df_nodes[df_nodes['source_label'] == target_pc]['source'].values[0]
metrics = ['geo_distance', 'inverse_trips']

for metric in metrics:
    for covid_phase, (title, df_covid_phase) in enumerate(zip(list_titles, list_df)):
        df, G = plot_shortest_path_tree(
            SOURCE, metric, covid_phase, df_covid_phase, df_distances, df_nodes, 
            title, 'Plaça Catalunya', want_labels=False)

## 3.7. Closeness centraliy <a class="anchor" id="closeness"></a>

In [ ]:
# Closeness of a node
target_mitma = '0801903'
SOURCE = df_nodes[df_nodes['source_label'] == target_mitma]['source'].values[0]

for metric in metrics:
    print("\n", metric)
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        closeness = round(nx.closeness_centrality(G, u=SOURCE, distance=metric, wf_improved=True), 8)
        print("\t", title, closeness)

In [ ]:
# Closeness distributions
metrics = [None, 'inverse_trips', 'geo_distance']

for metric in metrics:
   
    # Create figure
    fig, axs = plt.subplots(2,5, figsize=(14, 6), sharey=True, sharex=True)
    fig.suptitle(f'Closeness centrality {metric}', size=16)
    row, column = 0, 0

    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):

        # Row and columns
        if covid_phase > 4:
            row = 1
        if covid_phase == 5:
            column = 0
        
        # Compute closeness
        closeness_dict = nx.closeness_centrality(G, distance=metric, wf_improved=True)
        df_clos = pd.DataFrame(closeness_dict.values(), columns=['closeness']).reset_index()
        
        # Plot histogram
        df_clos['closeness'].hist(ax=axs[row][column])
        
        # Plot mean data vertical line
        mean = df_clos['closeness'].mean()
        axs[row][column].axvline(x=mean, color='b')
        
        # Title
        axs[row][column].set_title(title, size=8)
        axs[row][column].annotate(
            xycoords='axes fraction', xy=(0.53, 0.9), 
            text=f"mean: {str(round(mean, 5))}", size=7.5)

        # Update column
        column+=1
    
    plt.tight_layout()
    plt.show()

In [ ]:
metrics = ['geo_distance']

for metric in metrics:
   
    # Create figure
    fig, axs = plt.subplots(2,5, figsize=(14, 6), sharey=True, sharex=True)
    fig.suptitle(f'Closeness centrality {metric}', size=16)
    row, column = 0, 0

    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):

        # Row and columns
        if covid_phase > 4:
            row = 1
        if covid_phase == 5:
            column = 0
        
        # Compute closeness
        closeness_dict = nx.closeness_centrality(G, distance=metric, wf_improved=True)
        df_clos = pd.DataFrame(closeness_dict.values(), columns=['closeness']).reset_index()
        
        # Plot histogram
        df_clos['closeness'].hist(ax=axs[row][column])
        
        # Plot mean data vertical line
        mean = df_clos['closeness'].mean()
        axs[row][column].axvline(x=mean, color='b')
        
        # Title
        axs[row][column].set_title(title, size=8)
        
        if metric != 'geo_distance':
            axs[row][column].annotate(
                xycoords='axes fraction', xy=(0.53, 0.9), 
                text=f"mean: {str(round(mean, 5))}", size=7.5)
        else:
            axs[row][column].annotate(
                xycoords='axes fraction', xy=(0.05, 0.9), 
                text=f"mean: {str(round(mean, 7))}", size=7.5)

        # Update column
        column+=1
    
    plt.tight_layout()
    plt.show()

## 3.8. Betweenness centraliy <a class="anchor" id="betweenness"></a>

In [ ]:
metrics = ['geo_distance']
target_mitma = '0801903'
SOURCE = df_nodes[df_nodes['source_label'] == target_mitma]['source'].values[0]

for metric in metrics:
    print("\n", metric)
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        betweenness_dict = nx.betweenness_centrality(G, normalized=True, weight=metric, endpoints=False, seed=None)
        print("\t", title, round(betweenness_dict[SOURCE], 6))

In [ ]:
metrics =  [None, 'inverse_trips', 'geo_distance']

for metric in metrics:
   
    # Create figure
    fig, axs = plt.subplots(2,5, figsize=(14, 6), sharey=True, sharex=True)
    fig.suptitle(f'Betweenness centrality {metric}', size=16)
    row, column = 0, 0

    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):

        # Row and columns
        if covid_phase > 4:
            row = 1
        if covid_phase == 5:
            column = 0
        
        # Compute closeness
        betweenness_dict = nx.betweenness_centrality(G, normalized=True, weight=metric, endpoints=False, seed=None)
        df_bet = pd.DataFrame(betweenness_dict.values(), columns=['closeness']).reset_index()
        
        # Plot histogram
        df_bet['closeness'].hist(ax=axs[row][column])
        
        # Plot mean data vertical line
        mean = df_bet['closeness'].mean()
        axs[row][column].axvline(x=mean, color='b')
        
        # Title
        axs[row][column].set_title(title, size=8)
        axs[row][column].annotate(xycoords='axes fraction', xy=(0.53, 0.9), text=f"mean: {str(round(mean, 5))}", size=7.5)

        # Update column
        column+=1
    
    plt.tight_layout()
    plt.show()

## 3.9. Transitivity and Clustering Coefficient ($C_{ij}$) <a class="anchor" id="clustering"></a>

In [ ]:
def compute_clustering(G, metric, weighted=True, compute_nodes_clustering=False):
    """
    Computes transitivity, clustering and average clustering
    """
    # Fraction of all possible triangles present in G
    transitivity = nx.transitivity(G)
    
    if weighted:
        if compute_nodes_clustering:
            # Compute the clustering coefficient for nodes
            nodes_clustering = nx.clustering(G, weight=metric)
        
        # Average clustering
        average_clustering = nx.average_clustering(G, weight=metric, count_zeros=True)
        print(f"Weighted average clustering: {round(average_clustering,3)}\n")
    
    else:
        if compute_nodes_clustering:
            # Compute the clustering coefficient for nodes
            nodes_clustering = nx.clustering(G)
        
        # Average clustering
        average_clustering = nx.average_clustering(G, count_zeros=True)
        print(f"Average clustering: {round(average_clustering,3)}\n")
        
    return transitivity, nodes_clustering, average_clustering

In [ ]:
metrics = ['trips', 'geo_distance']

for metric in metrics:
    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        print(title)
        file = f'{save_url}/{phases_names[covid_phase]}_dict_nodes_clustering_{metric}.pkl'

        if exists(file):
            continue
        else:
            transitivity, nodes_clustering, average_clustering = compute_clustering(
                G, metric, weighted=True, compute_nodes_clustering=True)
            with open(file,"wb") as f:
                pickle.dump(nodes_clustering, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Plot clustering
metrics = ['trips', 'geo_distance']

for metric in metrics: 

    # Create figure
    fig, axs = plt.subplots(2,5, figsize=(14, 6), sharey=True, sharex=True)
    fig.suptitle(f'Clustering coefficient {metric}', size=16)
    row, column = 0, 0

    for covid_phase, (G, title) in enumerate(zip(list_G, list_titles)):
        # Transitivity
        print(f"Transitivity {metric}")
        transitivity = round(nx.transitivity(G),2 )
        print(title, transitivity)
        
        # Clustering plot
        # Row and columns
        if covid_phase > 4:
            row = 1
        if covid_phase == 5:
            column = 0

        # Open data and plot it
        with open(f'{save_url}/{phases_names[covid_phase]}_dict_nodes_clustering_{metric}.pkl', 'rb') as handle:
            b = pickle.load(handle)
        df = pd.DataFrame(list(b.values()), columns=['clustering'])
        df.hist(ax=axs[row][column])

        # Plot mean data vertical line
        mean = df['clustering'].mean()
        axs[row][column].axvline(x=mean, color='b')

        # Title
        axs[row][column].set_title(title, size=8)
        axs[row][column].annotate(
            xycoords='axes fraction', xy=(0.53, 0.9), text=f"mean: {str(round(mean, 2))}", size=7.5)

        # Update column
        column+=1

    plt.show()